<a href="https://colab.research.google.com/github/SaraWestWA/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/SW_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
# look at dataframe

print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [6]:
# non-numeric information

df.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,23040,23040,23040,23039,23039,23040,5201,22987,23040,23040
unique,5,11,44,10,146,22691,1724,3652,147,120
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,D4,100 JEROME STREET,4,0,D4,01/24/2019
freq,7494,19117,5061,11071,3408,6,99,7500,3408,480


In [7]:
# numeric information

df.describe(include='number')

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [8]:
df['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS                       5061
02 TWO FAMILY DWELLINGS                       4567
10 COOPS - ELEVATOR APARTMENTS                3471
13 CONDOS - ELEVATOR APARTMENTS               3339
03 THREE FAMILY DWELLINGS                     1438
07 RENTALS - WALKUP APARTMENTS                 807
09 COOPS - WALKUP APARTMENTS                   672
15 CONDOS - 2-10 UNIT RESIDENTIAL              421
04 TAX CLASS 1 CONDOS                          418
44 CONDO PARKING                               366
17 CONDO COOPS                                 300
22 STORE BUILDINGS                             288
05 TAX CLASS 1 VACANT LAND                     288
12 CONDOS - WALKUP APARTMENTS                  256
14 RENTALS - 4-10 UNIT                         200
29 COMMERCIAL GARAGES                          147
08 RENTALS - ELEVATOR APARTMENTS               120
30 WAREHOUSES                                  105
21 OFFICE BUILDINGS                             96
31 COMMERCIAL VACANT LAND      

In [9]:
# narrow price range

df=df[(df['SALE_PRICE']>100000) & (df['SALE_PRICE']<2000000)]
df.shape

(13411, 21)

In [10]:
# drop rows which are not in the correct building class

df=df[df['BUILDING_CLASS_CATEGORY'].str.contains('01 ONE FAMILY DWELLINGS')]
df.shape

(3151, 21)

In [11]:
df.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,3151,3151,3151,3151,3151,3151,1,3151,3151,3151
unique,5,7,1,2,13,3135,1,1035,11,91
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,94 CELESTE COURT,RP.,"4,000",A1,01/31/2019
freq,1580,2959,3151,3111,1185,2,1,289,1186,78


In [12]:
# change the sale date column to a timestamp

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format = True)

In [13]:
# turn land area into a number
df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .str.replace(',','')
    .astype(int)
  ) 

In [14]:
df.describe(include='number')

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,3151.000000,3151.000000,0.0,3151.000000,3151.000000,3151.000000,3151.000000,3151.000000,3151.000000,3151.000000,3151.0,3.151000e+03
mean,6917.976515,75.981593,NaN,11027.219613,0.987623,0.015868,1.003491,3112.117423,1470.306887,1943.694700,1.0,6.285601e+05
std,3963.326705,161.089514,NaN,482.875284,0.113414,0.127499,0.171789,1790.055434,586.339200,26.676786,0.0,2.929904e+05
min,21.000000,1.000000,NaN,10030.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.040000e+05
25%,4016.000000,21.000000,NaN,10461.000000,1.000000,0.000000,1.000000,2000.000000,1144.000000,1925.000000,1.0,4.475000e+05
50%,6301.000000,42.000000,NaN,11235.000000,1.000000,0.000000,1.000000,2542.000000,1360.000000,1938.000000,1.0,5.680000e+05
75%,10208.500000,69.000000,NaN,11413.000000,1.000000,0.000000,1.000000,4000.000000,1683.000000,1955.000000,1.0,7.600000e+05
max,16350.000000,2720.000000,NaN,11697.000000,2.000000,2.000000,3.000000,18906.000000,7875.000000,2018.000000,1.0,1.955000e+06


In [15]:
# Test / Train Split
cutoff = pd.to_datetime('2019-04-01')
train = df[df['SALE_DATE'] < cutoff]
test  = df[df['SALE_DATE']>= cutoff]

# verify split
print(train.shape)
print(test.shape)

(2507, 21)
(644, 21)


In [16]:
# easement column is entirely filled with Nan
# exclude extraneous
print(df['EASE-MENT'].value_counts(dropna=False),'\n')

# TAX_CLASS_AT_TIME_OF_SALE column all one value
# exclude, no variation
print(df['TAX_CLASS_AT_TIME_OF_SALE'].value_counts(dropna=False),'\n')

# check out residential units variation
# definition from NYC Department of Finance "Residential Units:
#The number of residential units at the listed property."
#exclude, all should be at least 1, only one 2
print(df['RESIDENTIAL_UNITS'].value_counts(dropna=False),'\n')

# check out commercial units variation
print(df['COMMERCIAL_UNITS'].value_counts(dropna=False),'\n')

# check out total units variation
# include, note: unless empty lots are included 0 units are incorrect
print(df['TOTAL_UNITS'].value_counts(dropna=False),'\n')

NaN    3151
Name: EASE-MENT, dtype: int64 

1    3151
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64 

1.0    3110
0.0      40
2.0       1
Name: RESIDENTIAL_UNITS, dtype: int64 

0.0    3102
1.0      48
2.0       1
Name: COMMERCIAL_UNITS, dtype: int64 

1.0    3061
2.0      49
0.0      40
3.0       1
Name: TOTAL_UNITS, dtype: int64 



In [17]:
list(df)

['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BLOCK',
 'LOT',
 'EASE-MENT',
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 'APARTMENT_NUMBER',
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'LAND_SQUARE_FEET',
 'GROSS_SQUARE_FEET',
 'YEAR_BUILT',
 'TAX_CLASS_AT_TIME_OF_SALE',
 'BUILDING_CLASS_AT_TIME_OF_SALE',
 'SALE_PRICE',
 'SALE_DATE']

In [33]:
#define features and target
target = 'SALE_PRICE'

# identify high cardinality and extranous columns
# tax class at present is 99% the one option and 1% the other
# 'RESIDENTIAL_UNITS', 'TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE'


extraneous = ['EASE-MENT','BUILDING_CLASS_CATEGORY','APARTMENT_NUMBER','TAX_CLASS_AT_TIME_OF_SALE','SALE_DATE']

high_cardinality = ['ADDRESS']

features = df.columns.drop([target] + extraneous + high_cardinality)

features

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [34]:
# create features matrix and target vector

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE
44,3,OTHER,1,5495,801,A9,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,A9
61,4,OTHER,1,7918,72,A1,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,A1
78,2,OTHER,1,4210,19,A1,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,A1
108,3,OTHER,1,5212,69,A1,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,A1
111,3,OTHER,1,7930,121,A5,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,A5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,1,4081,44,A2,10306.0,1.0,0.0,1.0,2400,921.0,1950.0,A2
18130,5,OTHER,1,2373,201,A5,10314.0,1.0,0.0,1.0,2450,2128.0,1980.0,A5
18132,5,OTHER,1,1132,42,A1,10302.0,1.0,0.0,1.0,4361,1807.0,2018.0,A1
18134,5,OTHER,1,3395,37,A2,10305.0,1.0,0.0,1.0,6000,621.0,1930.0,A2


In [35]:
# one hot encoding

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

print (X_train.shape, X_test.shape)
print (X_train.dtypes)
X_train

(2507, 47) (644, 47)
BOROUGH_3                              int64
BOROUGH_4                              int64
BOROUGH_2                              int64
BOROUGH_5                              int64
BOROUGH_1                              int64
NEIGHBORHOOD_OTHER                     int64
NEIGHBORHOOD_FLUSHING-NORTH            int64
NEIGHBORHOOD_EAST NEW YORK             int64
NEIGHBORHOOD_BEDFORD STUYVESANT        int64
NEIGHBORHOOD_FOREST HILLS              int64
NEIGHBORHOOD_BOROUGH PARK              int64
NEIGHBORHOOD_ASTORIA                   int64
TAX_CLASS_AT_PRESENT_1                 int64
TAX_CLASS_AT_PRESENT_1D                int64
BLOCK                                  int64
LOT                                    int64
BUILDING_CLASS_AT_PRESENT_A9           int64
BUILDING_CLASS_AT_PRESENT_A1           int64
BUILDING_CLASS_AT_PRESENT_A5           int64
BUILDING_CLASS_AT_PRESENT_A0           int64
BUILDING_CLASS_AT_PRESENT_A2           int64
BUILDING_CLASS_AT_PRESENT_A3      

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,0,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,0,1,0,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,0,1,0,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0,0,0,1,0,1,0,0,0,0,0,0,1,0,4081,44,0,0,0,0,1,0,0,0,0,0,0,0,0,10306.0,1.0,0.0,1.0,2400,921.0,1950.0,0,0,0,0,1,0,0,0,0,0,0
18130,0,0,0,1,0,1,0,0,0,0,0,0,1,0,2373,201,0,0,1,0,0,0,0,0,0,0,0,0,0,10314.0,1.0,0.0,1.0,2450,2128.0,1980.0,0,0,1,0,0,0,0,0,0,0,0
18132,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1132,42,0,1,0,0,0,0,0,0,0,0,0,0,0,10302.0,1.0,0.0,1.0,4361,1807.0,2018.0,0,1,0,0,0,0,0,0,0,0,0
18134,0,0,0,1,0,1,0,0,0,0,0,0,1,0,3395,37,0,0,0,0,1,0,0,0,0,0,0,0,0,10305.0,1.0,0.0,1.0,6000,621.0,1930.0,0,0,0,0,1,0,0,0,0,0,0


In [39]:
# select K best, start with 12 features

from sklearn.feature_selection import SelectKBest, f_regression

# change to 29 features to see final selection
selector = SelectKBest(k=29)

X_train_selected = selector.fit_transform(X_train,y_train)
X_test_selected = selector.transform(X_test)

X_train_selected.shape, X_test_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


((2507, 29), (644, 29))

In [40]:
# make lists of selected features

select_mask = selector.get_support()
select_mask

all_names = X_train.columns
select_names = all_names[select_mask]
nonselect_names = all_names[~select_mask]

print('Features used')
for name in select_names:
  print(name)
print('\n')


print('Features not used')
for name in nonselect_names:
  print(name)

Features used
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_PRESENT_B3
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
YEAR_BUILT
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A4
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_A8
BUILDING_CLASS_AT_TIME_OF_SALE_S0


Features not used
BOROUGH_3
BOROUGH_4
BOROUGH_2
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
BLOCK
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_

In [41]:
# use sklearn to to decide how many features to use

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range (1, len(X_train.columns)+1):
  print(f'{k} features')
  
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred =  model.predict(X_test_selected)
  mae = mean_absolute_error(y_test,y_pred)
  print(f'test mean absolute error: ${mae:,.0f} \n')

1 features
test mean absolute error: $183,641 

2 features
test mean absolute error: $179,555 

3 features
test mean absolute error: $179,291 

4 features
test mean absolute error: $179,291 

5 features
test mean absolute error: $178,897 

6 features
test mean absolute error: $177,128 

7 features
test mean absolute error: $171,378 

8 features
test mean absolute error: $171,465 

9 features
test mean absolute error: $170,925 

10 features
test mean absolute error: $167,468 

11 features
test mean absolute error: $167,182 

12 features
test mean absolute error: $166,712 

13 features
test mean absolute error: $156,488 

14 features
test mean absolute error: $157,546 

15 features
test mean absolute error: $157,543 

16 features
test mean absolute error: $157,545 

17 features
test mean absolute error: $157,406 

18 features
test mean absolute error: $157,406 

19 features
test mean absolute error: $157,282 

20 features
test mean absolute error: $157,282 

21 features
test mean absolut

In [79]:
# Ridge regression

from sklearn.linear_model import Ridge
for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
  model = Ridge (alpha=alpha, normalize = True)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  alpha=alpha
# Get linear equation
  m = model.coef_.round(2)
  b = model.intercept_.round(2)
  mae = mean_absolute_error(y_test, y_pred)

  print(f'alpha={alpha}, mae:{mae:.0f}')

alpha=0.001, mae:154898
alpha=0.01, mae:154500
alpha=0.1, mae:153815
alpha=1.0, mae:167591
alpha=1, mae:167591
alpha=100.0, mae:209979
alpha=1000.0, mae:211401


In [81]:
# Refine farther
# alpha = 0.08, mae $153,748 is the best, which does not beat the non-ridge linear model

for alpha in [0.01, 0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]:
  model = Ridge (alpha=alpha, normalize = True)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  alpha=alpha
# Get linear equation
  m = model.coef_.round(2)
  b = model.intercept_.round(2)
  mae = mean_absolute_error(y_test, y_pred)

  print(f'alpha={alpha}, mae:{mae:.0f}')

alpha=0.01, mae:154500
alpha=0.02, mae:154270
alpha=0.03, mae:154107
alpha=0.04, mae:153967
alpha=0.05, mae:153858
alpha=0.06, mae:153795
alpha=0.07, mae:153763
alpha=0.08, mae:153748
alpha=0.09, mae:153772
